# 2D Mode solver

## Importing packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import A_FMM

In [ ]:
a = 1.5
ay = 1.0

s = 0.300
t = 0.050
w = 0.4
N=10

cr = A_FMM.Creator()
cr.ridge(12.0, 2.0, 2.0, w/a, s/ay, t/ay)
lay = A_FMM.Layer(N,N, creator=cr, Nyx=ay/a) 
fig, ax = plt.subplots(1,2, figsize=(14, 4))

x, y = np.linspace(-1.0, 1.0, 201), np.linspace(-1.0, 1.0, 201)

eps = lay.calculate_epsilon(x=x, y=y)
_=ax[0].contourf(a*x, a*y, np.squeeze(eps['eps']).T, levels=41)
fig.colorbar(_, ax=ax[0], label='Epsilon')
ax[0].set_title('Without coordinate trasfomrmation')

lay.transform(ex=0.6, ey=0.6)
eps = lay.calculate_epsilon(x=x, y=y)
_=ax[1].contourf(a*x, a*y, np.squeeze(eps['eps']).T, levels=41)
fig.colorbar(_, ax=ax[1], label='Epsilon')
ax[1].set_title('With coordinate trasfomrmation')

for _ in ax:
    _.set_xlabel('x'), _.set_ylabel('y')
plt.tight_layout()

In [ ]:
lay.mode(a/1.55)
fig, ax = plt.subplots(1,4, figsize=(16, 4))
for i in range(4):
    print(f'Mode{i} effective index: {index[i]}')
    u = lay.create_input({i:1.0})
    field = lay.calculate_field(u, x=x, y=y)
    Ex = np.squeeze(field['Ex'])
    Ey = np.squeeze(field['Ey'])
    ax[i].set_title(f'Mode{i}')
    _=ax[i].contourf(a*x, a*y, abs(Ex.T) ** 2.0 + abs(Ey.T) **2.0, levels=41)
    ax[i].set_xlabel('x')
    ax[i].set_ylabel('y')
    fig.colorbar(_, ax=ax[i], label='|E|^2')
plt.tight_layout()

### Covergence test

In [ ]:
a = 4.0
ay = 2.0

def create_layer(N, s=1.0, t=0.1, w=3.0, a=a, ay=ay):
    cr = A_FMM.Creator()
    cr.ridge(3.44**2.0, 3.40**2.0, 1.0, w/a, s/ay, t/ay)
    lay = A_FMM.Layer(N,N, creator=cr, Nyx=ay/a) 
    lay.transform(ex=0.6, ey=0.6)
    return lay

In [ ]:
N_list = [5, 10, 15, 20, 25]
BTE, BTM = [], []
BTE_target = 0.3019
BTM_target = 0.2674
fig, ax = plt.subplots(1, 5 , figsize=(14, 4))
i = 0
for N in N_list:
    lay = create_layer(N)
    lay.mode(a/1.15)
    n_te = lay.get_index()[0]
    n_tm = lay.get_index()[1]
    Bte = (n_te**2.0 - 3.4**2.0) / (3.44**2.0 - 3.4**2.0)
    Btm = (n_tm**2.0 - 3.4**2.0) / (3.44**2.0 - 3.4**2.0)
    BTE.append(Bte)
    BTM.append(Btm)


ax[0].plot(N_list, [abs(_ - BTE_target) for _ in BTE], '-o')
ax[0].plot(N_list, [abs(_ - BTM_target) for _ in BTM], '-o')
ax[0].set_yscale('log')
ax[0].grid()
